# Chapter 10: Cluster Randomization and Hierarchical Modeling 聚类随机化与分层建模

# 代码来源：
https://github.com/BuissonFlorent/BehavioralDataAnalysis/tree/master/Chapter%2010%20-%20Cluster%20Randomization%20and%20Hierarchical%20Modeling
# 本实验验证：
新的SOP让顾客在互动过程中有更好体验，即获得更高的CSAT（每月顾客平均满意度）测量值。
# 在设计A/B测试时，回答以下问题：
1. 需要多少样本量才能有80%的把握检测到效应？
2. 给定样本量，能检测到的最小效应是多少？
3. 应该使用什么置信水平？
# 典型工作流程：
1. 使用历史数据估计变异性和基线值
2. 设定目标效应大小（有意义的最小效应）
3. 运行模拟计算不同参数组合下的功效
4. 选择满足功效要求的实验设计

# Libraries and data

### Libraries

In [1]:
# Common libraries
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
import seaborn as sns

# Chapter-specific libraries
# To rescale numeric variables
from sklearn.preprocessing import MinMaxScaler
# To one-hot encode cat. variables 分类数据数值化
from sklearn.preprocessing import OneHotEncoder

### Data
数据包括
- 呼叫中心的Center_ID
- 客服ID Rep_ID
- 顾客年龄Age
- 致电原因Reason
- 顾客满意度 CALL_CSAT
- 所属组别grp，如对照组和实验组，仅实验数据有，历史数据无
处理数据
- 调整中心ID以区别于索引
- 移除不需要的变量

In [2]:
hist_data_df = pd.read_csv('chap10-historical_data.csv')
exp_data_df = pd.read_csv('chap10-experimental_data.csv')

#Shifting center_ID to distinguish it from indices 调整中心ID以区别于索引
hist_data_df['center_ID'] = hist_data_df['center_ID'] + 100 
exp_data_df['center_ID'] = exp_data_df['center_ID'] + 100 

#Removing the variable M6Spend we won't use in this chapter 移除不需要的变量
del(hist_data_df['M6Spend'])
del(exp_data_df['M6Spend'])

# Introduction to hierarchical modeling 分层建模

分层模型适用于解决多重共线性问题，也即一个分类变量依赖于另一个分类变量，即嵌套分类变量。分层模型将组视为从可能无限的组分布中随机抽取的组。

In [3]:
# Hierarchical analysis of historical data with center ID only as clustering variable
# 仅以中心ID作为聚类变量的历史数据层次分析 原因和年龄 混合线性模型回归结果

mixed = smf.mixedlm("call_CSAT ~ reason + age", data = hist_data_df, 
                   groups = hist_data_df["center_ID"])
print(mixed.fit().summary())

            Mixed Linear Model Regression Results
Model:              MixedLM Dependent Variable: call_CSAT    
No. Observations:   695205  Method:             REML         
No. Groups:         10      Scale:              1.1217       
Min. group size:    54203   Log-Likelihood:     -1026427.7247
Max. group size:    79250   Converged:          Yes          
Mean group size:    69520.5                                  
-------------------------------------------------------------
                   Coef. Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------------
Intercept          3.899    0.335  11.641 0.000  3.243  4.556
reason[T.property] 0.199    0.003  74.786 0.000  0.194  0.205
age                0.020    0.000 176.747 0.000  0.020  0.020
Group Var          1.122    0.407                            



In [4]:
# Hierarchical analysis of historical data with center ID and rep ID as clustering variable
# 以中心ID和代表ID为聚类变量的历史数据分层分析 混合线性模型回归结果 代表ID是嵌套在中心ID变量下的一个聚类变量。
# 以方差和标准差的形式测量呼叫中心内部和呼叫中心之间数据的可变性。
# 固定效应表示呼叫层次变量的系数
vcf = {"rep_ID": "0+C(rep_ID)"} #在方差分量公式中表示低层嵌套变量
mixed2 = smf.mixedlm("call_CSAT ~ reason + age", 
                    data = hist_data_df, 
                    groups = hist_data_df["center_ID"],
                    re_formula='1',
                    vc_formula=vcf)
print(mixed2.fit().summary())

            Mixed Linear Model Regression Results
Model:             MixedLM  Dependent Variable:  call_CSAT   
No. Observations:  695205   Method:              REML        
No. Groups:        10       Scale:               0.3904      
Min. group size:   54203    Log-Likelihood:      -660424.9323
Max. group size:   79250    Converged:           Yes         
Mean group size:   69520.5                                   
-------------------------------------------------------------
                   Coef. Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------------
Intercept          3.901    0.367  10.639 0.000  3.182  4.620
reason[T.property] 0.200    0.002 126.789 0.000  0.196  0.203
age                0.020    0.000 298.302 0.000  0.020  0.020
Group Var          1.304    0.976                            
rep_ID Var         0.776    0.131                            



# Determining random assignment and sample size/power 确定随机分配与样本量/检验效能

假设最小可检测效应是0.6，我们的判定规则得出实验组确实比对照组更好的概率是多少？


In [5]:
# Functions for simulation # 模拟函数
# 从分层模型返回实验组的系数
# 采用标准功效分析统计公式，T检验公式
def hlm_metric_fun(dat_df):
    vcf = {"rep_ID": "0+C(rep_ID)"}
    h_mod = smf.mixedlm("call_CSAT ~ reason + age + grp", 
                   data = dat_df, 
                  groups = dat_df["center_ID"],
                    re_formula='1',
                   vc_formula=vcf)
    coeff = h_mod.fit().fe_params.values[2]
    return coeff


# 通过自助法计算指定指标的置信区间
#通过有放回的重抽样模拟数据生成过程，从原始数据中多次抽样，每次计算统计量，用这些统计量的分布来估计真实参数的分布
def boot_CI_fun(dat_df, metric_fun = hlm_metric_fun, B = 20, conf_level = 9/10):
    Ncalls_rep = 1200
    coeff_boot = []
    # Calculate coeff of interest for each simulation #计算每次模拟中目标系数
    for b in range(B):
        boot_df = dat_df.groupby("rep_ID").sample(n=Ncalls_rep,
                                                  replace=True).\
        reset_index(drop= True)
        coeff = metric_fun(boot_df)
        coeff_boot.append(coeff)
    #Extract confidence interval #提取置信区间
    coeff_boot.sort()
    offset = round(B * (1 - conf_level) / 2)
    confint = [coeff_boot[offset], coeff_boot[-(offset+1)]]
    return confint

# 基于置信区间做出二元决策 
# 决策为1：有足够的证据表明指标显著大于0
# 决策为0：没有足够证据表明指标显著大于0
def decision_fun(dat_df, metric_fun, B = 20, conf_level = 0.9):
    boot_CI = boot_CI_fun(dat_df, metric_fun, B = B, conf_level = conf_level)
    decision = 1 if boot_CI[0] > 0  else 0
    return decision

## Random assignment

在呼叫中心层级随机化，根据呼叫中心的特征分层，例如客服代表数量和呼叫指标的平均值。

In [6]:
# Aggregating data to center level # 将数据聚合至中心层级
center_data_df = hist_data_df.groupby('center_ID').agg(
        nreps = ('rep_ID', lambda x: x.nunique()),
        avg_call_CSAT = ("call_CSAT", "mean"),
        avg_age=("age", "mean"),
        pct_reason_pmt=('reason', 
                        lambda x: sum(1 if r=='payment' else 0 for r in x)/len(x))
        )

#Reformatting variables as needed # 按需重新格式化变量
center_data_df['nreps'] = center_data_df.nreps.astype(float)

center_data_df.head()

,nreps,avg_call_CSAT,avg_age,pct_reason_pmt
center_ID,,,,
101,18.0,3.664430,39.962880,0.601027
102,21.0,3.958169,39.959532,0.599237
103,22.0,4.030376,39.981830,0.599508
104,15.0,5.296561,40.063354,0.599690
105,21.0,5.921405,39.977681,0.600679


In [7]:
### Function to prep the data 准备数据的函数
def strat_prep_fun(dat_df):

    #Extracting components of the data  #提取数据组件
    num_df = dat_df.copy().loc[:,dat_df.dtypes=='float64'] #Numeric vars
    center_ID = [i for i in dat_df.index]

    #Normalizing all numeric variables to [0,1] #将所有数值变量归一化至[0,1]区间
    scaler = MinMaxScaler()
    scaler.fit(num_df)
    num_np = scaler.transform(num_df)
    
    return center_ID, num_np

# K-近邻匹配算法，采用贪心策略，
# 在观测研究中创建匹配的控制组
def pair_fun(dat_df, K = 2):
    
    match_len = K - 1 # Number of matches we want to find 需寻找的匹配对数量
    match_idx = match_len - 1 # Accounting for 0-indexing 考虑0索引规则
    
    center_ID, data_np = strat_prep_fun(dat_df)
    N = len(data_np)
    
    #Calculate distance matrix 计算距离矩阵
    from scipy.spatial import distance_matrix
    d_mat = distance_matrix(data_np, data_np)
    np.fill_diagonal(d_mat,N+1)
    # Set up variables 设置变量
    available = [i for i in range(N)]
    available_temp = available.copy()
    matches_lst = []
    lim = int(N/match_len)
    
    closest = np.argpartition(d_mat, kth=match_idx,axis=1)
    
    for n in available:
        if len(matches_lst) == lim: break
        if n in available_temp:
            for match_lim in range(match_idx,N-1):
                possible_matches = closest[n,:match_lim].tolist()
                matches = list(set(available_temp) & set(possible_matches))
                if len(matches) == match_len:
                    matches.append(n)
                    matches_lst.append(matches)
                    available_temp \
                    = [m for m in available_temp if m not in matches]
                    break
                else:
                    closest[n,:] = np.argpartition(d_mat[n,:], kth=match_lim)
    #Map center indices to their proper IDs #将中心索引映射至对应ID
    matches_id_lst = [[center_ID[k[0]],center_ID[k[1]]] for k in matches_lst]
    return np.array(matches_id_lst)
pair_fun(center_data_df)

array([[102, 101],
       [106, 103],
       [107, 104],
       [110, 105],
       [109, 108]])

## Power analysis

功效分析模拟函数，用于评估统计检验检测给定效应大小的能力。

In [8]:
##### Simulation function 模拟函数#####
def power_sim_fun(dat_df, metric_fun = hlm_metric_fun, Ncalls_rep = 1000, eff_size = 1, B = 20, conf_level = 0.9):
    """
    功能：模拟A/B测试并计算统计功效
    参数：
      - dat_df: 包含实验数据的DataFrame
      - metric_fun: 用于计算效应量的函数，默认使用hlm_metric_fun
      - Ncalls_rep: 每组的重抽样数量，默认1000
      - eff_size: 模拟的效应大小（加到处理组的call_CSAT上），默认1
      - B: 自助法重抽样次数，默认20
      - conf_level: 置信水平，默认0.9（90%）
    返回：包含每次模拟决策结果的列表（1=检测到效应，0=未检测到）
    """ 
    # 不同参数对功效的影响：
    # 1. eff_size：效应大小越大，功效越高
    # 2. Ncalls_rep：样本量越大，功效越高
    # 3. B：自助法迭代次数越多，置信区间估计越稳定
    # 4. conf_level：置信水平越高，检测要求越严格，功效越低
    #Extract the stratified pairs 提取分层配对数据
    stratified_pairs = stratified_assgnt_fun(dat_df, K=2)
    Npairs = len(stratified_pairs)
    Nperm = 2 ** Npairs
    power_list = []
    
    for m in dat_df.month.unique():
        #Sample down the data 抽样数据
        sample_data_df = dat_df.loc[dat_df.month==m,]
        sample_data_df = sample_data_df.groupby('rep_ID')\
        .sample(n=Ncalls_rep, replace=True)\
        .reset_index(drop = True)
        # 遍历所有可能的分配方案，完全随机化检验，在每种分配下，添加预设的效应大小，检验统计方法是否能检测到这个已知效应
        for perm in range(Nperm):
            bin_str = f'{perm:0{Npairs}b}'
            idx = np.array([[i for i in range(Npairs)],
                            [int(d) for d in bin_str]]).T
            treat = [stratified_pairs[tuple(idx[i])] for i in range(Npairs)]
            
            sim_data_df = sample_data_df.copy()
            sim_data_df['group'] = 'ctrl'
            sim_data_df.loc[(sim_data_df.center_ID.isin(treat)),'group']\
                = 'treat'
            
            sim_data_df.loc[(sim_data_df.group=='treat'),'call_CSAT'] =\
                sim_data_df.loc[(sim_data_df.group=='treat'),'call_CSAT'] + eff_size
                
            sim_data_df.loc[(sim_data_df.call_CSAT > 10), 'call_CSAT'] = 10

            # 统计决策
            # Option 1: extract CIs for visualization 方案1：提取置信区间用于可视化
            #sim_CI = boot_CI_fun(sim_data_df, lm_metric_fun)
            #power_list.append(sim_CI)
            
            # Option 2: calculate decision for overall power determination 方案2：计算整体功效判定决策
            D = decision_fun(sim_data_df, metric_fun, B = B, conf_level = conf_level)
            power_list.append(D)
            
    return power_list

# Analyzing the experiment 分析实验
用度量函数分析实验数据，获得度量值的Bootstrap 90%-CI

In [9]:
##### Analysis of experimental data #####

coeff = hlm_metric_fun(exp_data_df)
print(coeff)

hlm_CI = boot_CI_fun(exp_data_df, hlm_metric_fun)
print(hlm_CI)

0.47790322079110353
[np.float64(0.47474867983408076), np.float64(0.4815910175343983)]


# 说明：
置信区间非常窄，在0.25以上。根据功效分析，真实效应值不太可能在这个CI内,而是可能更低,也可能更高,预期效应值等于0.48，高于决策阈值，将实施干预，尽管预期效应值小于目标效应。置信区间比根据正态近似得到的置信区间(即系数+/-1.96*系数标准误差)小得多，部分原因在于分层随机化。